In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

from sklearn.model_selection import GridSearchCV
from  sklearn.tree  import  DecisionTreeClassifier
from  sklearn.ensemble  import  RandomForestClassifier , VotingClassifier
from  sklearn.linear_model  import  LogisticRegression
from  sklearn.metrics  import  accuracy_score , roc_curve , auc , f1_score, confusion_matrix, classification_report
from  sklearn.preprocessing  import  LabelEncoder , MinMaxScaler
from  sklearn  import svm #SVC , LinearSVC
from sklearn.neighbors import NearestNeighbors
#draw roc cuver
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from imblearn.metrics import (geometric_mean_score, make_index_balanced_accuracy)
from imblearn.metrics import classification_report_imbalanced

import cloud
import classifier

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

dt = cloud.read_data_sets()

In [ ]:
num_epoch = 5000

batch_size = 100
z_dim = 10
X_dim = dt.train.features.shape[1]
y_dim = dt.train.labels.shape[1]
h_dim = 25
c = 0
lr = 1e-3

X_train = dt.train.features
Y_train = dt.train.labels

X_test = dt.test.features
Y_test = dt.test.labels
num_sample = len (X_train)


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


# =============================== Q(z|X) ======================================

X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, y_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

Q_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
Q_b2 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X, c):
    inputs = tf.concat(axis=1, values=[X, c])
    h1 = tf.nn.relu(tf.matmul(inputs, Q_W1) + Q_b1)
    h2 = tf.nn.relu(tf.matmul(h1, Q_W2) + Q_b2)
    z_mu = tf.matmul(h2, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h2, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps


# =============================== P(X|z) ======================================

P_W1 = tf.Variable(xavier_init([z_dim + y_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W3 = tf.Variable(xavier_init([h_dim, h_dim]))
P_b3 = tf.Variable(tf.zeros(shape=[h_dim]))


P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    h = tf.nn.relu(tf.matmul(inputs, P_W1) + P_b1)
    h3 = tf.nn.relu(tf.matmul(h, P_W3) + P_b3)
    logits = tf.matmul(h3, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits




In [ ]:
X_train = dt.train.features
Y_train = dt.train.labels

X_test = dt.test.features
Y_test = dt.test.labels

In [ ]:
print ("-------------------------LOI TREN TAP TRAIN-------ORIGINAL---------------")


print ("SVM")
acc = classifier.svm(X_train, Y_train, X_train, Y_train)
print ("Decision tree")
acc = classifier.decisiontree(X_train, Y_train, X_train, Y_train)
print ("Random forest")
acc= classifier.randomforest(X_train, Y_train, X_train, Y_train)

In [ ]:
print ("-------------------------LOI TREN TAP TEST--ORIGINAL--------------------")


print ("SVM")
acc = classifier.svm(X_train, Y_train, X_test,Y_test)
print ("Decision tree")
acc = classifier.decisiontree(X_train, Y_train, X_test,Y_test)
print ("Random forest")
acc= classifier.randomforest(X_train, Y_train, X_test,Y_test)

In [ ]:
# =============================== TRAINING ====================================

z_mu, z_logvar = Q(X, c)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample, c)

# Sampling from random z
X_samples, _ = P(z, c)

# E[log P(X|z)]
recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
# D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
# VAE loss
vae_loss = tf.reduce_mean(recon_loss + kl_loss)

solver = tf.train.AdamOptimizer(learning_rate = lr).minimize(vae_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#if not os.path.exists('out/'):
#    os.makedirs('out/')

i = 0
for epoch in range(num_epoch):
    for iter in range(num_sample // batch_size):
        X_mb, y_mb = dt.train.next_batch(batch_size)

        _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb, c: y_mb})

    if epoch % 20 == 0:
        print('Iter: {}'.format(epoch))
        print('Loss: {:.4}'. format(loss))
        print()

        

In [ ]:
z_train = sess.run(z_sample, feed_dict={X: X_train, c: Y_train})
z_test = sess.run (z_sample, feed_dict = {X: X_test, c: Y_test})


In [ ]:
#nslkdd
n_samples = 1000
y = np.zeros(shape=[n_samples, y_dim])
idx = 1
y[:, idx] = 1.
X_gen0 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen0 = y



In [ ]:
import ll_parzen
#combine Generated samples and Original Samples

#cloud
X = np.concatenate ((X_train, X_gen0), axis = 0)
Y = np.concatenate ((Y_train, Y_gen0), axis = 0)


from collections import Counter
Y_count = np.argmax(Y, axis=1)
print('Resampled dataset shape {}'.format(Counter(Y_count)))


s = np.arange(X.shape[0])
np.random.shuffle(s)
X = X[s]
Y = Y[s]

print ("-------------------------CVAE----------------------")


print ("SVM")
acc = classifier.svm(X, Y, X_test,Y_test)
print ("Decision tree")
acc= classifier.decisiontree(X, Y, X_test,Y_test)
print ("Random forest")
auc = classifier.randomforest(X, Y, X_test,Y_test)

In [ ]:
n_samples = 35000
y = np.zeros(shape=[n_samples, y_dim])
idx = 0
y[:, idx] = 1.
X_gen0 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen0 = y

n_samples = 35000
y = np.zeros(shape=[n_samples, y_dim])
idx = 1
y[:, idx] = 1.
X_gen1 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen1 = y

n_samples = 33000
y = np.zeros(shape=[n_samples, y_dim])
idx = 2
y[:, idx] = 1.
X_gen2 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen2 = y

n_samples = 4000
y = np.zeros(shape=[n_samples, y_dim])
idx = 3
y[:, idx] = 1.
X_gen3 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen3 = y

n_samples = 21000
y = np.zeros(shape=[n_samples, y_dim])
idx = 4
y[:, idx] = 1.
X_gen4 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen4 = y

n_samples = 27000
y = np.zeros(shape=[n_samples, y_dim])
idx = 7
y[:, idx] = 1.
X_gen5 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen5 = y

n_samples = 36000
y = np.zeros(shape=[n_samples, y_dim])
idx = 8
y[:, idx] = 1.
X_gen6 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen6 = y

n_samples = 37000
y = np.zeros(shape=[n_samples, y_dim])
idx = 9
y[:, idx] = 1.
X_gen7 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen7 = y

n_samples = 19000
y = np.zeros(shape=[n_samples, y_dim])
idx = 5
y[:, idx] = 1.
X_gen8 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen8 = y

n_samples = 20000
y = np.zeros(shape=[n_samples, y_dim])
idx = 3
y[:, idx] = 1.
X_gen9 = sess.run(X_samples, feed_dict={z: np.random.randn(n_samples, z_dim), c: y})
Y_gen9 = y
